In [2]:
import os
import pandas as pd

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur CSV
csv_path = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final.csv')

# 3. Einlesen
df = pd.read_csv(csv_path, encoding='utf-8')

# 4. Count vor dem Löschen
print(f"Zeilen vor Löschen von 'mueller': {len(df)}")

# 5. Alle Zeilen löschen, in denen in der Spalte COMPONENT_cleaned 'mueller' vorkommt
mask = df['COMPONENT_cleaned'].str.contains('mueller', case=False, na=False)
df_filtered = df.loc[~mask]

# 6. Count nach dem Löschen
print(f"Zeilen nach Löschen von 'mueller': {len(df_filtered)}")

# 7. Neues CSV speichern
out_path = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final_nomueller.csv')
df_filtered.to_csv(out_path, index=False, encoding='utf-8')
print(f"Gefiltertes CSV gespeichert als: {out_path}")



Zeilen vor Löschen von 'mueller': 6072
Zeilen nach Löschen von 'mueller': 6071
Gefiltertes CSV gespeichert als: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/lexikon_deDE15LinguisticVariant_final_nomueller.csv


In [11]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur aktuellen CSV (passe ggf. den Dateinamen an)
csv_in  = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final.csv')
csv_out = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final_noarticles.csv')

# 3. Einlesen
df = pd.read_csv(csv_in, encoding='utf-8')

# 4. Liste der zu entfernenden Wörter (alles in Kleinbuchstaben)
articles = ['stunden', 'nach', 'und', 'die', 'der', 'das', 'in', 'mit', 'zu', 'auf', 'einer', 'ordnung', 'fuer', 'sie', 'normal', 'an', 'vor', 'auch']

# 5. Funktion, die die Artikel und überflüssige Leerzeichen löscht
def remove_articles(text):
    if pd.isna(text):
        return text
    text = text.lower()
    # Regex für ganze Wort-Matches der Artikel
    pattern = r'\b(?:' + '|'.join(articles) + r')\b'
    # Entfernen und mehrfach Leerzeichen auf ein reduzieren
    text = re.sub(pattern, ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 6. Spalte aktualisieren
df['COMPONENT_cleaned'] = df['COMPONENT_cleaned'].apply(remove_articles)

# 7. Speichern
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Alle Artikel entfernt – neues CSV: {csv_out}")


Alle Artikel entfernt – neues CSV: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/lexikon_deDE15LinguisticVariant_final_noarticles.csv


In [12]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfade für Eingabe- und Ausgabe-CSV
csv_in  = os.path.join(base_dir, 'lexikon_ATC-Bedeutung_final.csv')
csv_out = os.path.join(base_dir, 'lexikon_ATC-Bedeutung_final_noarticles.csv')

# 3. CSV einlesen
df = pd.read_csv(csv_in, encoding='utf-8')

# 4. Liste der zu entfernenden Wörter (alles in Kleinbuchstaben)
articles = ['stunden', 'nach', 'und', 'die', 'der', 'das', 'in', 'mit', 'zu', 'auf', 'einer', 'ordnung', 'fuer', 'sie', 'normal', 'an', 'vor', 'auch']

# 5. Funktion, die die Artikel und überflüssige Leerzeichen löscht
def remove_articles(text):
    if pd.isna(text):
        return text
    text = text.lower()
    pattern = r'\b(?:' + '|'.join(articles) + r')\b'
    text = re.sub(pattern, ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 6. Spalte updaten
df['ATC-Bedeutung_cleaned'] = df['ATC-Bedeutung_cleaned'].apply(remove_articles)

# 7. Neues CSV speichern
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Neues CSV ohne Artikel gespeichert: {csv_out}")


Neues CSV ohne Artikel gespeichert: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/lexikon_ATC-Bedeutung_final_noarticles.csv


In [3]:
import os
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# 1. Falls noch nicht geschehen: Stopwords herunterladen
nltk.download('stopwords')

# 2. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 3. Ein- und Ausgabepfade
csv_in  = os.path.join(base_dir, 'lexikon_ATC-Bedeutung_final.csv')
csv_out = os.path.join(base_dir, 'lexikon_ATC-Bedeutung_final_noarticles.csv')

# 4. CSV einlesen
df = pd.read_csv(csv_in, encoding='utf-8')

# 5. Deutsche Stopwords + eigene Liste zusammenführen
stops = set(stopwords.words('german'))
stops.update([
    'stunden','nach','und','die','der','das','in','mit','zu','auf',
    'einer','ordnung','fuer','sie','normal','an','vor','auch',
    'andere','ohne'
])

# 6. Reinigungsfunktion
def filter_stopwords(text):
    if pd.isna(text):
        return text
    # in lowercase
    tokens = re.findall(r'\b[a-zäöüß]+\b', text.lower())
    # herausfiltern
    filtered = [t for t in tokens if t not in stops and len(t) > 1]
    return ' '.join(filtered)

# 7. Spalte updaten
df['ATC-Bedeutung_cleaned'] = df['ATC-Bedeutung_cleaned'].apply(filter_stopwords)

# 8. Speichern
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Fertig – neue Datei: {csv_out}")



Fertig – neue Datei: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/lexikon_ATC-Bedeutung_final_noarticles.csv


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mosimacnew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
from nltk.corpus import stopwords
stops = stopwords.words('german')
print(len(stops), "Stoppwörter geladen")
print(stops[:20])


232 Stoppwörter geladen
['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders']


In [5]:
import os
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# 1. Falls noch nicht geschehen: Stopwords herunterladen
nltk.download('stopwords')

# 2. Basis-Verzeichnis
base_dir = os.getcwd()

# 3. Ein-/Ausgabepfade
csv_in  = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final.csv')
csv_out = os.path.join(base_dir, 'lexikon_deDE15LinguisticVariant_final_noarticles.csv')

# 4. CSV einlesen
df = pd.read_csv(csv_in, encoding='utf-8')

# 5. Stopword-Set aus NLTK + eigene Ergänzungen
stops = set(stopwords.words('german'))
stops.update([
    'stunden','nach','und','die','der','das','in','mit','zu','auf',
    'einer','ordnung','fuer','sie','normal','an','vor','auch',
    'andere','ohne'
])

# 6. Reinigungsfunktion
def clean_component(text):
    if pd.isna(text):
        return text
    text = text.lower()
    # Tokenisieren auf Wortgrenzen (inkl. Umlaute)
    tokens = re.findall(r'\b[a-zäöüß]+\b', text)
    # Filter: keine Stopwords, keine Einbuchstaben-Wörter
    filtered = [t for t in tokens if t not in stops and len(t) > 1]
    return ' '.join(filtered)

# 7. Spalte anwenden
df['COMPONENT_cleaned'] = df['COMPONENT_cleaned'].apply(clean_component)

# 8. Speichern
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Neues CSV ohne Stopwords und Einbuchstaben: {csv_out}")



Neues CSV ohne Stopwords und Einbuchstaben: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/lexikon_deDE15LinguisticVariant_final_noarticles.csv


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mosimacnew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
